<a href="https://colab.research.google.com/github/dw5189/openai-gemini/blob/main/%E4%BC%98%E5%8C%96%E5%90%8E%E7%9A%84HIC%E8%AE%A1%E7%AE%97%E5%99%A8%E4%BB%A3%E7%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate # 导入了，但当前版本主要使用 numpy.trapezoid
from numpy import trapezoid
from numpy.random import normal
import warnings
from typing import Tuple, Dict, List, Optional

# --- 5. 配置管理 ---
class HICConfig:
    """HIC 计算和可视化配置设置。"""
    DEFAULT_TIME_WINDOW = 15 # 默认的 HIC 时间窗口，单位为毫秒
    INTERPOLATION_POINTS = 1000 # 插值边界数据时使用的点数
    NOISE_LEVEL = 0.05 # 合成加速度的噪声水平（例如，平均加速度的 5%）
    OUTPUT_DPI = 300 # 保存图表时的 DPI
    MAX_ACCELERATION = 1000 # 最大合理加速度值 (g)
    MIN_TIME_STEP = 0.001 # 最小时间步长 (毫秒)
    MIN_WINDOW_POINTS = 2 # 时间窗口内最少数据点数

    # Matplotlib 中文显示字体设置
    FONT_SANS_SERIF = ['SimHei'] # 用于正常显示中文标签
    AXES_UNICODE_MINUS = False # 解决负号显示问题

# 应用 Matplotlib 中文显示配置
plt.rcParams['font.sans-serif'] = HICConfig.FONT_SANS_SERIF
plt.rcParams['axes.unicode_minus'] = HICConfig.AXES_UNICODE_MINUS

# 定义上边界和下边界的加速度-时间数据
# 时间单位为毫秒，加速度单位为 g
UPPER_BOUNDARY_DATA = {
    'time': [0, 6, 44, 58],
    'acceleration': [0.5, 25.5, 25.5, 0]
}

LOWER_BOUNDARY_DATA = {
    'time': [2, 13, 40, 48],
    'acceleration': [0, 18.5, 18.5, 0]
}

# --- 3. 增强代码结构 ---
class HICCalculator:
    """HIC计算器类 - 提供完整的HIC计算、验证和可视化功能"""

    def __init__(self, time_window: float = HICConfig.DEFAULT_TIME_WINDOW):
        """
        初始化HIC计算器

        参数:
        time_window: 时间窗口大小 (毫秒)
        """
        self.time_window = time_window # 时间窗口在毫秒
        self.config = HICConfig()

    def validate_input(self, time: np.ndarray, accel: np.ndarray) -> bool:
        """
        验证输入数据的有效性

        参数:
        time: 时间数组 (毫秒)
        accel: 加速度数组 (g)

        返回:
        bool: 数据是否有效
        """
        try:
            # 检查数组长度
            if len(time) != len(accel):
                raise ValueError("时间和加速度数组长度不匹配")

            if len(time) < 2:
                raise ValueError("数据点数量不足，至少需要2个点")

            # 检查时间序列单调性
            if not np.all(np.diff(time) > 0):
                raise ValueError("时间序列必须严格递增")

            # 检查数值范围
            if np.any(time < 0):
                raise ValueError("时间值不能为负")

            # 处理负加速度值
            if np.any(accel < 0):
                warnings.warn("检测到负加速度值，将被设为0")
                accel = np.maximum(accel, 0)

            if np.any(accel > self.config.MAX_ACCELERATION):
                warnings.warn(f"检测到异常大的加速度值 (>{self.config.MAX_ACCELERATION}g)")

            # 检查时间步长
            min_dt = np.min(np.diff(time))
            if min_dt < self.config.MIN_TIME_STEP:
                warnings.warn(f"时间步长过小 ({min_dt}ms < {self.config.MIN_TIME_STEP}ms)")

            return True

        except Exception as e:
            print(f"输入验证失败: {e}")
            return False

    def interpolate_data(self, data: Dict[str, List], num_points: Optional[int] = None) -> Tuple[np.ndarray, np.ndarray]:
        """
        对加速度-时间数据进行线性插值

        参数:
        data: 包含'time'和'acceleration'键的字典
        num_points: 插值点数，默认使用配置值

        返回:
        time_dense: 密集时间数组 (毫秒)
        accel_dense: 密集加速度数组 (g)
        """
        if num_points is None:
            num_points = self.config.INTERPOLATION_POINTS

        time_array = np.array(data['time'], dtype=float) # 确保为浮点类型
        accel_array = np.array(data['acceleration'], dtype=float) # 确保为浮点类型

        # 验证输入
        if not self.validate_input(time_array, accel_array):
            raise ValueError("输入数据验证失败")

        time_dense = np.linspace(np.min(time_array), np.max(time_array), num_points)
        accel_dense = np.interp(time_dense, time_array, accel_array)

        return time_dense, accel_dense

    def calculate_hic(self, time: np.ndarray, accel: np.ndarray) -> Tuple[float, float, float]:
        """
        计算给定时间窗口的最大HIC值 - 修复版本（迭代搜索）

        参数:
        time: 时间数组 (毫秒)
        accel: 加速度数组 (g)

        返回:
        max_hic: 最大HIC值
        t1_max: 最大HIC对应的起始时间 (毫秒)
        t2_max: 最大HIC对应的结束时间 (毫秒)
        """
        # 验证输入
        if not self.validate_input(time, accel):
            raise ValueError("HIC计算输入数据验证失败")

        max_hic = 0.0
        t1_max, t2_max = 0.0, 0.0

        # 转换时间单位为秒，用于积分计算
        time_sec = time / 1000.0

        n = len(time)
        # 迭代搜索算法
        for i in range(n):
            t1 = time[i]

            # 找到时间窗口内的所有有效结束点
            # 使用 np.searchsorted 进行优化，尽管外部仍是循环
            end_time_for_window_sec = time_sec[i] + self.time_window / 1000.0
            j_end_idx = np.searchsorted(time_sec, end_time_for_window_sec, side='right')

            for j in range(i + 1, min(j_end_idx, n)):
                t2 = time[j]

                dt_ms = t2 - t1

                # 检查时间窗口约束
                if dt_ms <= 0 or dt_ms > self.time_window:
                    continue

                # 选取时间窗口内的数据点
                window_indices = np.arange(i, j + 1)

                # 确保有足够的点进行积分
                if len(window_indices) < self.config.MIN_WINDOW_POINTS:
                    continue

                window_time_sec = time_sec[window_indices]
                window_accel = accel[window_indices]

                try:
                    # 计算积分
                    integral = trapezoid(window_accel, window_time_sec)

                    # 计算HIC值
                    dt_sec = dt_ms / 1000.0 # 将毫秒转换为秒

                    # HIC公式：[(1/T)∫a(t)dt]^2.5 * T
                    avg_accel = integral / dt_sec
                    hic = (avg_accel ** 2.5) * dt_sec

                    # 更新最大HIC值
                    if hic > max_hic:
                        max_hic = hic
                        t1_max = t1
                        t2_max = t2

                except Exception as e:
                    # 捕获可能出现的数值问题，如dt_sec接近0导致的除零
                    warnings.warn(f"HIC计算警告 (t1={t1:.2f}, t2={t2:.2f}, 错误: {e})")
                    continue

        return max_hic, t1_max, t2_max

    def calculate_hic_vectorized(self, time: np.ndarray, accel: np.ndarray) -> Tuple[float, float, float]:
        """
        向量化的HIC计算方法 - 性能优化版本 (部分向量化)

        注意：此方法仍包含内层循环，若要完全向量化需更复杂的Numpy广播技巧。
        当前的“向量化”主要体现在切片和对小窗口数据使用numpy函数。

        参数:
        time: 时间数组 (毫秒)
        accel: 加速度数组 (g)

        返回:
        max_hic: 最大HIC值
        t1_max: 最大HIC对应的起始时间 (毫秒)
        t2_max: 最大HIC对应的结束时间 (毫秒)
        """
        # 验证输入
        if not self.validate_input(time, accel):
            raise ValueError("向量化HIC计算输入数据验证失败")

        n = len(time)
        time_sec = time / 1000.0 # 转换为秒

        max_hic = 0.0
        t1_max, t2_max = 0.0, 0.0

        # 遍历所有可能的起始点 (t1)
        for i in range(n):
            t1_ms = time[i]
            t1_sec = time_sec[i]

            # 确定当前窗口允许的最大结束时间点
            end_time_for_window_sec = t1_sec + self.time_window / 1000.0

            # 使用 searchsorted 高效找到所有可能结束点的索引范围
            # j_start_idx 确保 t2 > t1 (j > i)
            # j_end_idx 确保 t2 - t1 <= self.time_window
            j_start_idx = i + 1
            j_end_idx = np.searchsorted(time_sec, end_time_for_window_sec, side='right')

            # 遍历当前 t1 的所有可能的结束点 (t2)
            for j in range(j_start_idx, min(j_end_idx, n)):
                t2_ms = time[j]
                t2_sec = time_sec[j]

                dt_ms = t2_ms - t1_ms
                dt_sec = t2_sec - t1_sec # 与 dt_ms / 1000.0 相同

                # 确保时间窗口有效且包含足够的数据点
                if dt_ms <= 0 or dt_ms > self.time_window or (j - i + 1) < self.config.MIN_WINDOW_POINTS:
                    continue

                # 直接切片获取窗口内的时间和加速度数据
                window_time_sec = time_sec[i : j + 1]
                window_accel = accel[i : j + 1]

                try:
                    # 向量化积分计算
                    integral = trapezoid(window_accel, window_time_sec)

                    # HIC计算
                    avg_accel = integral / dt_sec
                    hic = (avg_accel ** 2.5) * dt_sec

                    # 更新最大值
                    if hic > max_hic:
                        max_hic = hic
                        t1_max = t1_ms
                        t2_max = t2_ms

                except Exception as e:
                    warnings.warn(f"向量化HIC计算警告 (t1={t1_ms:.2f}, t2={t2_ms:.2f}, 错误: {e})")
                    continue

        return max_hic, t1_max, t2_max

    def generate_composite_acceleration(self, upper_time: np.ndarray, upper_accel: np.ndarray,
                                        lower_time: np.ndarray, lower_accel: np.ndarray,
                                        noise_level: Optional[float] = None) -> Tuple[np.ndarray, np.ndarray]:
        """
        生成物理上更真实的合成加速度曲线

        参数:
        upper_time: 上边界时间数组 (毫秒)
        upper_accel: 上边界加速度数组 (g)
        lower_time: 下边界时间数组 (毫秒)
        lower_accel: 下边界加速度数组 (g)
        noise_level: 噪声水平，默认使用配置值

        返回:
        composite_time: 合成加速度的时间数组 (毫秒)
        composite_accel: 合成加速度数组 (g)
        """
        if noise_level is None:
            noise_level = self.config.NOISE_LEVEL

        # 确定时间范围
        min_time = max(np.min(upper_time), np.min(lower_time))
        max_time = min(np.max(upper_time), np.max(lower_time))

        # 创建统一的时间数组
        composite_time = np.linspace(min_time, max_time, self.config.INTERPOLATION_POINTS)

        # 在统一时间点上插值上下边界加速度
        upper_accel_interp = np.interp(composite_time, upper_time, upper_accel)
        lower_accel_interp = np.interp(composite_time, lower_time, lower_accel)

        # 使用更物理的模型生成合成加速度
        # 1. 基于时间的权重函数（模拟冲击过程）
        time_normalized = (composite_time - min_time) / (max_time - min_time)
        # 这是一个示例的权重函数，可以根据需要调整其形状
        weight_function = 0.5 + 0.3 * np.sin(2 * np.pi * time_normalized) * np.exp(-2 * time_normalized)

        # 2. 生成基础合成加速度
        base_accel = lower_accel_interp + weight_function * (upper_accel_interp - lower_accel_interp)

        # 3. 添加时间相关的平滑噪声 (模拟传感器漂移或低频振动)
        dt = (max_time - min_time) / len(composite_time)
        freq_scale = 1.0 / (dt * 10) # 控制噪声频率，确保它是低频
        smooth_noise = np.zeros_like(composite_time)

        for i in range(1, 4): # 添加多个频率分量
            phase = np.random.random() * 2 * np.pi
            amplitude = noise_level * np.mean(base_accel) / i # 随频率衰减的幅度
            smooth_noise += amplitude * np.sin(2 * np.pi * freq_scale * i * composite_time + phase)

        # 4. 添加高频随机噪声 (模拟测量白噪声)
        random_noise = normal(0, noise_level * np.mean(base_accel) * 0.1, len(composite_time))

        # 5. 合成最终加速度
        composite_accel = base_accel + smooth_noise + random_noise

        # 确保加速度值非负且合理
        composite_accel = np.maximum(composite_accel, 0)
        composite_accel = np.minimum(composite_accel, self.config.MAX_ACCELERATION)

        return composite_time, composite_accel

    def visualize(self, results: Dict, save_path: Optional[str] = None) -> None:
        """
        可视化HIC计算结果

        参数:
        results: 包含计算结果的字典
        save_path: 保存路径，默认为None（不保存）
        """
        plt.figure(figsize=(12, 8))

        # 绘制原始数据点
        if 'upper_boundary' in results:
            plt.plot(results['upper_boundary']['time'], results['upper_boundary']['acceleration'],
                     'bo', label='上边界数据点', markersize=8)

        if 'lower_boundary' in results:
            plt.plot(results['lower_boundary']['time'], results['lower_boundary']['acceleration'],
                     'ro', label='下边界数据点', markersize=8)

        # 绘制插值曲线
        if 'upper_interpolated' in results:
            plt.plot(results['upper_interpolated']['time'], results['upper_interpolated']['accel'],
                     'b-', label='上边界曲线', linewidth=2)

        if 'lower_interpolated' in results:
            plt.plot(results['lower_interpolated']['time'], results['lower_interpolated']['accel'],
                     'r-', label='下边界曲线', linewidth=2)

        # 绘制合成加速度曲线
        if 'composite' in results:
            plt.plot(results['composite']['time'], results['composite']['accel'],
                     'g--', label='模拟合成加速度曲线', alpha=0.7, linewidth=1.5)

        # 标记最大HIC对应的时间窗口
        if 'upper_hic' in results:
            upper_hic = results['upper_hic']
            plt.axvspan(upper_hic['t1'], upper_hic['t2'], alpha=0.2, color='blue',
                        label=f'上边界最大HIC窗口: {upper_hic["value"]:.2f}')

        if 'lower_hic' in results:
            lower_hic = results['lower_hic']
            plt.axvspan(lower_hic['t1'], lower_hic['t2'], alpha=0.2, color='red',
                        label=f'下边界最大HIC窗口: {lower_hic["value"]:.2f}')

        if 'composite_hic' in results:
            composite_hic = results['composite_hic']
            plt.axvspan(composite_hic['t1'], composite_hic['t2'], alpha=0.2, color='green',
                        label=f'合成曲线最大HIC窗口: {composite_hic["value"]:.2f}')

        # 设置图表属性
        plt.title('SISA滑动座椅加速度边界与HIC计算', fontsize=16, fontweight='bold')
        plt.xlabel('时间 (毫秒)', fontsize=14)
        plt.ylabel('加速度 (g)', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.legend(fontsize=12)

        # 优化布局
        plt.tight_layout()

        # 保存图表
        if save_path:
            plt.savefig(save_path, dpi=self.config.OUTPUT_DPI, bbox_inches='tight')
            print(f"图表已保存至: {save_path}")

        # 显示图表
        plt.show()

    def calculate_all(self, upper_data: Dict, lower_data: Dict,
                      use_vectorized: bool = False) -> Dict:
        """
        执行完整的HIC计算流程

        参数:
        upper_data: 上边界数据字典
        lower_data: 下边界数据字典
        use_vectorized: 是否使用向量化计算

        返回:
        results: 包含所有计算结果的字典
        """
        results = {}

        try:
            # 插值数据
            upper_time, upper_accel = self.interpolate_data(upper_data)
            lower_time, lower_accel = self.interpolate_data(lower_data)

            # 存储原始数据和插值结果
            results['upper_interpolated'] = {'time': upper_time, 'accel': upper_accel}
            results['lower_interpolated'] = {'time': lower_time, 'accel': lower_accel}
            results['upper_boundary'] = upper_data
            results['lower_boundary'] = lower_data

            # 选择计算方法
            calc_method = self.calculate_hic_vectorized if use_vectorized else self.calculate_hic

            # 计算HIC值
            upper_hic, upper_t1, upper_t2 = calc_method(upper_time, upper_accel)
            lower_hic, lower_t1, lower_t2 = calc_method(lower_time, lower_accel)

            # 存储HIC结果
            results['upper_hic'] = {'value': upper_hic, 't1': upper_t1, 't2': upper_t2}
            results['lower_hic'] = {'value': lower_hic, 't1': lower_t1, 't2': lower_t2}

            # 生成合成加速度曲线
            composite_time, composite_accel = self.generate_composite_acceleration(
                upper_time, upper_accel, lower_time, lower_accel
            )
            results['composite'] = {'time': composite_time, 'accel': composite_accel}

            # 计算合成加速度的HIC
            composite_hic, composite_t1, composite_t2 = calc_method(composite_time, composite_accel)
            results['composite_hic'] = {'value': composite_hic, 't1': composite_t1, 't2': composite_t2}

            return results

        except Exception as e:
            print(f"计算过程中发生错误: {e}")
            raise

# --- 辅助函数（保持向后兼容性，供main_legacy使用）---
def interpolate_data(data: Dict[str, List], num_points: int = HICConfig.INTERPOLATION_POINTS) -> Tuple[np.ndarray, np.ndarray]:
    """
    对加速度-时间数据进行线性插值。
    （向后兼容版本，建议使用HICCalculator实例的interpolate_data方法）

    参数:
        data (dict): 包含 'time' 和 'acceleration' 列表的字典。
        num_points (int): 插值点数。

    返回:
        tuple: (time_dense, accel_dense) 插值后的 NumPy 数组。
    """
    time_dense = np.linspace(min(data['time']), max(data['time']), num_points)
    accel_dense = np.interp(time_dense, data['time'], data['acceleration'])
    return time_dense, accel_dense

def generate_composite_acceleration(upper_time: np.ndarray, upper_accel: np.ndarray, lower_time: np.ndarray, lower_accel: np.ndarray,
                                    num_points: int = HICConfig.INTERPOLATION_POINTS, noise_level: float = HICConfig.NOISE_LEVEL) -> Tuple[np.ndarray, np.ndarray]:
    """
    通过在上下边界之间混合并添加真实噪声来生成合成加速度曲线。
    （向后兼容版本，建议使用HICCalculator实例的generate_composite_acceleration方法）

    参数:
        upper_time (np.ndarray): 上边界的时间数组（毫秒）。
        upper_accel (np.ndarray): 上边界的加速度数组（g）。
        lower_time (np.ndarray): 下边界的时间数组（毫秒）。
        lower_accel (np.ndarray): 下边界的加速度数组（g）。
        num_points (int): 合成曲线的点数。
        noise_level (float): 噪声的标准差，表示为平均加速度的一部分。

    返回:
        tuple: (composite_time, composite_accel) NumPy 数组（毫秒，g）。
    """
    # 确定插值的重叠时间范围
    min_overall_time = max(upper_time.min(), lower_time.min())
    max_overall_time = min(upper_time.max(), lower_time.max())

    # 在重叠范围内为合成曲线创建统一的时间数组
    composite_time = np.linspace(min_overall_time, max_overall_time, num_points)

    # 将上下边界加速度插值到合成时间数组上
    upper_accel_interp = np.interp(composite_time, upper_time, upper_accel)
    lower_accel_interp = np.interp(composite_time, lower_time, lower_accel)

    # 在边界之间混合：每个点的随机加权平均
    random_blend_factors = np.random.rand(num_points) # 0 到 1 之间的随机因子
    base_accel = lower_accel_interp + random_blend_factors * (upper_accel_interp - lower_accel_interp)

    # 根据基准加速度的平均幅度添加真实噪声
    noise_amplitude = noise_level * np.mean(base_accel)
    noise = normal(0, noise_amplitude, num_points)
    composite_accel = base_accel + noise

    # 确保加速度值为非负，因为此处不期望物理上为负的加速度
    composite_accel = np.maximum(composite_accel, 0)

    return composite_time, composite_accel


# --- 主执行函数 ---
def main():
    """主函数 - 演示HIC计算器的使用"""
    print("=== HIC计算器 - 优化版本 ===")
    print(f"时间窗口: {HICConfig.DEFAULT_TIME_WINDOW}ms")
    print(f"插值点数: {HICConfig.INTERPOLATION_POINTS}")
    print()

    try:
        # 创建HIC计算器实例
        # 实例化时传递时间窗口，并确保名称与HICConfig中的DEFAULT_TIME_WINDOW一致
        calculator = HICCalculator(time_window=HICConfig.DEFAULT_TIME_WINDOW)

        # 执行完整计算流程
        print("正在执行HIC计算...")
        # 默认使用非向量化版本作为基础，便于对比
        results = calculator.calculate_all(UPPER_BOUNDARY_DATA, LOWER_BOUNDARY_DATA, use_vectorized=False)

        # 打印结果
        print("\n=== 计算结果 ===")
        upper_hic = results['upper_hic']
        lower_hic = results['lower_hic']
        composite_hic = results['composite_hic']

        print(f"上边界最大HIC值: {upper_hic['value']:.2f}，时间窗口: {upper_hic['t1']:.2f}ms - {upper_hic['t2']:.2f}ms")
        print(f"下边界最大HIC值: {lower_hic['value']:.2f}，时间窗口: {lower_hic['t1']:.2f}ms - {lower_hic['t2']:.2f}ms")
        print(f"合成加速度最大HIC值: {composite_hic['value']:.2f}，时间窗口: {composite_hic['t1']:.2f}ms - {composite_hic['t2']:.2f}ms")

        # 性能对比测试
        print("\n=== 性能对比测试 ===")
        import time

        # 标准算法
        start_time = time.time()
        _ = calculator.calculate_all(UPPER_BOUNDARY_DATA, LOWER_BOUNDARY_DATA, use_vectorized=False)
        standard_time = time.time() - start_time

        # 向量化算法
        start_time = time.time()
        _ = calculator.calculate_all(UPPER_BOUNDARY_DATA, LOWER_BOUNDARY_DATA, use_vectorized=True)
        vectorized_time = time.time() - start_time

        print(f"标准算法耗时: {standard_time:.6f}秒") # 增加精度
        print(f"向量化算法耗时: {vectorized_time:.6f}秒") # 增加精度
        if vectorized_time > 0:
            print(f"性能提升: {standard_time/vectorized_time:.2f}倍")

        # 可视化结果
        print("\n正在生成可视化图表...")
        calculator.visualize(results, save_path='hic_calculation_result_optimized.png')

        print("\n=== 计算完成 ===")

    except Exception as e:
        print(f"计算过程中发生错误: {e}")
        import traceback
        traceback.print_exc()

# 向后兼容的主函数 (保留原始函数式接口，使用全局辅助函数和旧的HICCalculator.calculate方法)
def main_legacy():
    """向后兼容的主函数 - 使用原始函数式设计"""
    print("\n=== 使用传统函数式接口 (向后兼容) ===")

    # 插值边界数据 (使用全局辅助函数)
    upper_interp_time, upper_interp_accel = interpolate_data(UPPER_BOUNDARY_DATA)
    lower_interp_time, lower_interp_accel = interpolate_data(LOWER_BOUNDARY_DATA)

    # 存储原始和插值后的边界数据，用于可视化
    # 注意：这里的可视化数据结构与HICCalculator.visualize期望的不同
    boundary_visualization_data = {
        'upper': {'raw_time': np.array(UPPER_BOUNDARY_DATA['time']),
                  'raw_accel': np.array(UPPER_BOUNDARY_DATA['acceleration']),
                  'interp_time': upper_interp_time,
                  'interp_accel': upper_interp_accel},
        'lower': {'raw_time': np.array(LOWER_BOUNDARY_DATA['time']),
                  'raw_accel': np.array(LOWER_BOUNDARY_DATA['acceleration']),
                  'interp_time': lower_interp_time,
                  'interp_accel': lower_interp_accel}
    }

    # 实例化HICCalculator，虽然其方法在main_legacy中不是完全按照面向对象方式使用
    # 此处的HICCalculator实例将使用其内部的validate_input和calculate方法
    # 注意：这个实例不再有time_window_sec属性，但calculate方法已经被修改以适应新的time_window属性
    hic_calculator_legacy = HICCalculator(time_window=HICConfig.DEFAULT_TIME_WINDOW)

    # 计算上边界的 HIC (使用HICCalculator实例的calculate_hic方法)
    # 注意：main_legacy中如果需要调用calculate，则应使用HICCalculator.calculate_hic
    # 为了保持main_legacy逻辑与原始版本一致，我这里修改为调用新的calculate_hic
    upper_hic, upper_t1, upper_t2 = hic_calculator_legacy.calculate_hic(upper_interp_time, upper_interp_accel)
    print(f"上边界最大HIC值: {upper_hic:.2f}，时间窗口: {upper_t1:.2f}ms - {upper_t2:.2f}ms")

    # 计算下边界的 HIC
    lower_hic, lower_t1, lower_t2 = hic_calculator_legacy.calculate_hic(lower_interp_time, lower_interp_accel)
    print(f"下边界最大HIC值: {lower_hic:.2f}，时间窗口: {lower_t1:.2f}ms - {lower_t2:.2f}ms")

    # 生成并计算合成加速度曲线的 HIC (使用全局辅助函数)
    composite_time, composite_accel = generate_composite_acceleration(
        upper_interp_time, upper_interp_accel, lower_interp_time, lower_interp_accel
    )
    composite_hic, composite_t1, composite_t2 = hic_calculator_legacy.calculate_hic(composite_time, composite_accel)
    print(f"合成曲线最大HIC值: {composite_hic:.2f}，时间窗口: {composite_t1:.2f}ms - {composite_t2:.2f}ms")

    # 汇总 HIC 结果用于可视化
    # 注意：这里的hic_results结构需要调整以适应新的HICCalculator.visualize方法
    hic_results_for_visual = {
        'upper_hic': {'value': upper_hic, 't1': upper_t1, 't2': upper_t2},
        'lower_hic': {'value': lower_hic, 't1': lower_t1, 't2': lower_t2},
        'composite_hic': {'value': composite_hic, 't1': composite_t1, 't2': composite_t2},
        'upper_interpolated': {'time': upper_interp_time, 'accel': upper_interp_accel},
        'lower_interpolated': {'time': lower_interp_time, 'accel': lower_interp_accel},
        'composite': {'time': composite_time, 'accel': composite_accel},
        'upper_boundary': UPPER_BOUNDARY_DATA,
        'lower_boundary': LOWER_BOUNDARY_DATA
    }

    # 可视化结果 (使用HICCalculator实例的visualize方法)
    hic_calculator_legacy.visualize(hic_results_for_visual, save_path='hic_calculation_result_legacy.png')

if __name__ == "__main__":
    main() # 优先运行新版主函数
    # 如果需要测试旧版，可以调用 main_legacy()
    # main_legacy()